# Linear mixed-effects models
### Douglas Bates

As described in the last section a _mixed-effects model_ or, more simply, a _mixed model_ incorporates both _fixed-effects_ parameters and _random effects_.  The random effects are associated with the levels of one or more _grouping factors_, which typically are _experimental units_ or _observational units_, such as `subject` or `item`. When we have several observations on each of several subjects, say, we may expect to see subject-to-subject variability in the response but assessing this variability is not the primary purpose of the data analysis.  Instead we wish to control for this level of variability in assessing difference in experimental factors.

For example, the [`sleepstudy`](https://rdrr.io/cran/lme4/man/sleepstudy.html) dataset in the [`lme4`](https://github.com/lme4/lme4) package for `R` is from a study on the effect of sleep deprivation on reaction time.  A sample from the population of interest (long-distance truck drivers) had their average response time measured when they were on their regular sleep schedule and after one up to nine days of sleep deprivation (allowed only 3 hours per day in which to sleep).

In [ ]:
using LinearAlgebra, MixedModels, RCall
RCall.ijulia_setdevice(MIME("image/svg+xml"), width=7, height=5)

In [ ]:
R"""
require(lme4, quietly=TRUE)
require(lattice, quietly=TRUE)
xyplot(Reaction ~ Days | Subject, sleepstudy,
       type = c("g","p","r"), layout = c(9,2),
       index = function(x,y) coef(lm(y ~ x))[1],
       xlab = "Days of sleep deprivation",
       ylab = "Average reaction time (ms)",
       aspect = "xy")
"""

Each panel shows the data from one subject as well as a simple linear regression line fit to that subject's data only.  The panels are ordered by increasing intercept of the within-subject line rwo-wise, starting at the bottom left.  Some subjects, e.g. 310 and 309, have fast reaction times and are almost unaffected by the sleep deprivation.  Others, e.g. 337, start with slow reaction times which then increase substantially after sleep depreivation.

A suitable model for these data would include an intercept and slope for the "typical" subject and randomly distributed deviations from these values for each of the observed subjects.  The assumed distribution of the random effects vector is multivariate Gaussian with mean zero (because they represent deviations from the population parameters) and an unknown covariance matrix, $\Sigma$, to be estimated from the data.

Because $\Sigma$ is a covariance matrix it must be symmetric and be positive-definite, a condition that is similar to the requirement that a scalar variance must be positive.  In particular, a positive-definite matrix like $\Sigma$ has a "square root" in the sense that there is a matrix $\mathbf{L}$ such that $\Sigma=\mathbf{L}\mathbf{L}^\prime$.  (Multiplying by $\mathbf{L}^\prime$ instead of squaring $\mathbf{L}$ is necessary to ensure that the product is symmetric.)  In fact, there are several such matrices $\mathbf{L}$.  If we require that $\mathbf{L}$ is lower triangular and that its diagonal entries be positive, there is only one such matrix, which is called the lower (or left) Cholesky factor.

As shown later, some of the expressions for the likelihood can be simplified if any scale parameter in the distribution of the response, given the random effects, is incorporated into the covariance matrix of the random effects.  We define $\mathbf{\lambda}$ to be the lower triangular matrix with non-negative diagonal entries such that
$$
\Sigma=\sigma^2{\bf\lambda}{\bf\lambda}^\prime
$$

It helps to consider the example at this point to clarify these concepts.  Returning to the `sleepstudy` data shown above

In [ ]:
sleepstudy = rcopy(R"lme4::sleepstudy")

we fit a model with fixed effects for the intercept and the slope with respect to days of sleep deprivation and, possibly correlated, random effects for each of these coefficients by `Subject`.

In [ ]:
f1 = @formula Reaction ~ 1 + Days + (1+Days|Subject)
m1 = fit(MixedModel, f1, sleepstudy)

The "estimated" random effects from this model are eighteen vectors, one for each subject, and each of length two (deviation for the intercept and for the slope).  These are returned as a $2\times 18$ matrix.

In [ ]:
first(ranef(m1))

The fixed-effects coefficients are the typical values for the population - initial reaction time of about 250 ms. and about 10.5 ms. increase in reaction time per day of sleep deprivation.

In [ ]:
fixef(m1)

For this model the matrix $\mathbf{\lambda}$ is estimated as

In [ ]:
λ = first(m1.λ)

and the (maximum likelihood) estimate of $\sigma^2$, as shown in the "Variance components" table, is

In [ ]:
σ² = varest(m1)

Thus the (maximum likelihood) estimate of the covariance matrix $\Sigma$ is

In [ ]:
Σ = σ² * λ * λ'

The correlation shown in the "Variance components" table can be evaluated as

In [ ]:
Σ[2,1] / sqrt(Σ[1,1] * Σ[2,2])

## The Big Picture

Although it is tempting to construct the model on a per-subject basis it is ultimately easier to consider the entire set of responses and the collection of all of the random effects together.  There are two reasons for this.  First, the parameters must be estimated from the complete data set.  Second, in situations where there is more than one grouping factor for the random effects it may not be possible to partition the responses according to the grouping factor.  In the sleepstudy example the 180 observations can be partitioned into eighteen groups of ten observations on each of the eighteen subjects.  However, in an example we will consider below each observation is on one of 56 subjects and one of 32 items and those classifications are _crossed_.  That is, each subject is tested on each item and each item is tested on each subject.  (Well, that was the plan at least.  As often happens a few observations were erroneously recorded so the factors are not completely crossed in the data after cleaning.)

In any case, we write $\mathbf{b}$ for the complete random-effects vector (in this case a 36-dimensional vector formed from the $2\times 18$ matrix in _column-major_ order).

In [ ]:
b = vec(first(ranef(m1)))

In the model the unconditional distribution of the random variable $\mathcal{B}$ is
$$
\mathcal{B}\sim\mathcal{N}\left(\mathbf{0},\sigma^2\mathbf{\Lambda}\mathbf{\Lambda}^\prime\right)
$$
and the conditional distribution of the response vector, $\mathcal{Y}$, is
$$
(\mathcal{Y}|\mathcal{B}=\mathbf{b})\sim\mathcal{N}\left(\mathbf{X\beta}+\mathbf{Zb}, \sigma^2\mathbf{I}_n\right)
$$

The model matrix $\mathbf{X}$ for the fixed-effects has the usual form

In [ ]:
Int.(m1.X)  # display as Int to reduce clutter

but the model matrix $\mathbf{Z}$ for the random effects is very sparse.  That is, most of the entries in $\mathbf{Z}$ are zero.

In [ ]:
Int.(first(m1.reterms))

In practice $\mathbf{Z}$ is stored and manipulated as a special type of sparse matrix.

The matrix $\Lambda$ is block-diagonal consisting of 18 diagonal blocks of size $2\times 2$, each of which is a copy of $\lambda$.  It could be written as a [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product)

In [ ]:
Λ = kron(I(18), first(m1.λ))

but there is no need to actually construct $\mathbf{\Lambda}$.  It is completely determined by $\mathbf{\lambda}$.

## Spherical random effects

One of the many useful properties of the normal distribution is that a scalar normal distribution, $\mathcal{X}\sim\mathcal{N}(\mu,\sigma^2)$, can be expressed in terms of the _standard normal_ distribution, $\mathcal{Z}\sim\mathcal{N}(0,1)$ as
$$
\mathcal{X} = \mu + \sigma \mathcal{Z}
$$
A similar result holds for the multivariate normal distribution.  The random effects vector, $\mathcal{B}$, with distribution $\mathcal{N}(\mathbf{0},\Sigma)$ can be generated from a "spherical" random effects vector, $\mathcal{U}$, as
$$
\mathcal{B} = \Lambda \mathcal{U}\quad\mathrm{where}\quad\mathcal{U}\sim\mathcal{N}(\mathbf{0},\sigma^2\mathbf{I}_q)
$$
and $q$ is the dimension of the random-effects vector (36 in our example).

(Recall that a multivariate normal distribution with covariance matrix $\sigma^2\mathbf{I}$ is called a "spherical normal" because the contours of constant probability density are spheres.  The random effects vector $\mathcal{U}$ has such a spherical distribution.)

Now the conditional distribution of the response, given the random effects, can be written in terms of $\mathcal{U}$ as
$$
(\mathcal{Y}|\mathcal{U}=\mathbf{u})\sim\mathcal{N}\left(\mathbf{X\beta}+\mathbf{Z\Lambda u}, \sigma^2\mathbf{I}_n\right)
$$

The joint probability density for $\mathcal{Y}$ and $\mathcal{U}$ is the product of the conditional density of $\mathcal{Y}|\mathcal{U}=\mathbf{u}$ and the unconditional density of $\mathcal{U}$.
$$
\begin{aligned}
f_{\mathcal{Y},\mathcal{U}}(\mathbf{y},\mathbf{u})&= \frac{1}{(2\pi\sigma^2)^{n/2}}\exp\left(-\frac{\|\mathbf{y}-\mathbf{X\beta}-\mathbf{Z\Lambda u}\|^2}{2\sigma^2}\right)\,\frac{1}{(2\pi\sigma^2)^{q/2}}\exp\left(-\frac{\|\mathbf{u}\|^2}{2\sigma^2}\right)\\
&=\frac{1}{(2\pi\sigma^2)^{(n+q)/2}}\exp\left(-\frac{\|\mathbf{y}-\mathbf{X\beta}-\mathbf{Z\Lambda u}\|^2+\|\mathbf{u}\|^2}{2\sigma^2}\right)
\end{aligned}
$$

Evaluating the likelihood requires the marginal distribution of $\mathcal{Y}$.  This can be obtained by integrating the joint distribution, $f_{\mathcal{Y},\mathcal{U}}(\mathbf{y},\mathbf{u})$, evaluated at the observed $\mathbf{y}$, with respect to $\mathbf{u}$. There is an analytic solution to this integral.  To derive this solution, we first write the penalized sum of squared residuals in a somewhat unusual but very useful form.  Let $\mathbf{\theta}$ be the vector of parameters that determine $\mathbf{\lambda}$.

In [ ]:
show(m1.θ)

In this case, $\mathbf{\theta}$ consists if the elements of the lower triangle of $\mathbf{\lambda}$.  What we will show is that, given a value of $\mathbf{\theta}$ the maximum of the log-likelihood for that value of $\mathbf{\theta}$ and any value of $\mathbf{\beta}$ and $\sigma$ can be determined from a matrix decomposition, specifically a Cholesky decomposition shown below.  This is called _profiling_ the log likelihood.

For the purposes of the optimization the objective is on the [_deviance_](https://en.wikipedia.org/wiki/Deviance_(statistics)) scale, which is negative twice the log-likelihood.  A summary of the optimization can be obtained as

In [ ]:
m1.optsum

It required fewer than 60 evaluations of the objective function to obtain the maximum likelihood estimates of $\mathbf{\theta}$ and, with them, the estimates of all the other parameters.

To evaluate the log-likelihood we write the penalized sum of squared residuals in the joint density, $f_{\mathcal{Y},\mathcal{U}}(\mathbf{y,u})$, as
$$
\begin{aligned}
r^2_\mathbf{\theta}(\mathbf{u},\mathbf{\beta}) &=  \|\mathbf{y}-\mathbf{X\beta}-\mathbf{Z\Lambda_\theta u}\|^2+\|\mathbf{u}\|^2\\
&=\left\|\begin{bmatrix}
\mathbf{Z\Lambda}&\mathbf{X}&\mathbf{y}\\
\mathbf{I}_q&\mathbf{0}&\mathbf{0}
\end{bmatrix}\begin{bmatrix}-\mathbf{u}\\ -\mathbf{\beta} \\ 1\end{bmatrix}\right\|^2 \\
&= \begin{bmatrix}-\mathbf{u}&-\mathbf{\beta}&1\end{bmatrix}
\begin{bmatrix}
\mathbf{\Lambda}^\prime\mathbf{Z}^\prime\mathbf{Z\Lambda}+\mathbf{I} & \mathbf{\Lambda}^\prime\mathbf{Z}^\prime\mathbf{X} & \mathbf{\Lambda}^\prime\mathbf{Z}^\prime\mathbf{y} \\
\mathbf{X}^\prime\mathbf{Z\Lambda} & \mathbf{X}^\prime\mathbf{X} & \mathbf{X}^\prime\mathbf{y} \\
\mathbf{y}^\prime\mathbf{Z\Lambda} & \mathbf{y}^\prime\mathbf{X} & \mathbf{y}^\prime\mathbf{y}
\end{bmatrix}
\begin{bmatrix}-\mathbf{u}\\ -\mathbf{\beta} \\ 1\end{bmatrix}\\
&=
\begin{bmatrix}-\mathbf{u}&-\mathbf{\beta}&1\end{bmatrix}
\begin{bmatrix}
\mathbf{R}_{ZZ}^\prime & \mathbf{0} & \mathbf{0} \\
\mathbf{R}_{ZX}^\prime & \mathbf{R}_{XX}^\prime & \mathbf{0} \\
\mathbf{r}_{Zy}^\prime & \mathbf{r}_{Xy}^\prime & r_{yy}
\end{bmatrix}
\begin{bmatrix}
\mathbf{R}_{ZZ} & \mathbf{R}_{ZX} & \mathbf{r}_{Zy} \\
\mathbf{0} & \mathbf{R}_{XX} & \mathbf{r}_{Xy} \\
\mathbf{0} & \mathbf{0} & r_{yy}
\end{bmatrix}
\begin{bmatrix}-\mathbf{u}\\ -\mathbf{\beta} \\ 1\end{bmatrix}\\
&= \left\|
\begin{bmatrix}
\mathbf{R}_{ZZ} & \mathbf{R}_{ZX} & \mathbf{r}_{Zy} \\
\mathbf{0} & \mathbf{R}_{XX} & \mathbf{r}_{Xy} \\
\mathbf{0} & \mathbf{0} & r_{yy}
\end{bmatrix}
\begin{bmatrix}-\mathbf{u}\\ -\mathbf{\beta} \\ 1\end{bmatrix}\right\|^2\\
&=\|\mathbf{r}_{Zy}-\mathbf{R}_{ZX}\mathbf{\beta}-\mathbf{R}_{ZZ}\mathbf{u}\|^2+ \|\mathbf{r}_{Xy}-\mathbf{R}_{XX}\mathbf{\beta}\|^2 + r_{yy}^2\\
&=r_{yy}^2+\|\mathbf{R}_{XX}\mathbf{\beta}-\mathbf{r}_{Xy}\|^2+\|\mathbf{R}_{ZZ}\mathbf{u}+\mathbf{R}_{ZX}\mathbf{\beta}-\mathbf{r}_{Zy}\|^2
\end{aligned}
$$
where
$$
\mathbf{R}(\mathbf{\theta})=
\begin{bmatrix}
\mathbf{R}_{ZZ} & \mathbf{R}_{ZX} & \mathbf{r}_{Zy} \\
\mathbf{0} & \mathbf{R}_{XX} & \mathbf{r}_{Xy} \\
\mathbf{0} & \mathbf{0} & r_{yy}
\end{bmatrix}
$$
is the upper triangular, right Cholesky factor of the symmetric, positive definite matrix
$$
\begin{bmatrix}
\mathbf{\Lambda}^\prime\mathbf{Z}^\prime\mathbf{Z\Lambda}+\mathbf{I} & \mathbf{\Lambda}^\prime\mathbf{Z}^\prime\mathbf{X} & \mathbf{\Lambda}^\prime\mathbf{Z}^\prime\mathbf{y} \\
\mathbf{X}^\prime\mathbf{Z\Lambda} & \mathbf{X}^\prime\mathbf{X} & \mathbf{X}^\prime\mathbf{y} \\
\mathbf{y}^\prime\mathbf{Z\Lambda} & \mathbf{y}^\prime\mathbf{X} & \mathbf{y}^\prime\mathbf{y}
\end{bmatrix}
$$
The sub-matrices on the diagonal, $\mathbf{R}_{ZZ}$ and $\mathbf{R}_{XX}$, are upper triangular and $\mathbf{R}_{ZZ}$ is sparse.  In our example, $\mathbf{R}_{ZZ}$ is $36\times 36$ but the only non-zeros are the upper triangles of $18$ blocks of size $2\times 2$ along the diagonal.  Also, the diagonal elements are, by construction, positive.  Because $\mathbf{R}_{ZZ}$ is triangular its determinant, $|\mathbf{R}_{ZZ}|$, is the product of its diagonal elements which also must be positive.

Furthermore, we can see that, for a fixed value of $\mathbf{\theta}$ the minimum $r^2_\mathbf{\theta}(\mathbf{u},\mathbf{\beta})$ is $r_{yy}^2$ and the conditional estimate of $\mathbf{\beta}$ satisfies
$$
\mathbf{R}_{XX}\widehat{\mathbf{\beta}}(\mathbf{\theta})=\mathbf{r}_{Xy} .
$$
The conditional mode, $\tilde{\mathbf{u}}$, of $\mathcal{U}$ given $\mathcal{Y}=\mathbf{y}$ is the solution to
$$
\mathbf{R}_{ZZ}\tilde{\mathbf{u}}=\mathbf{r}_{Zy}-\mathbf{R}_{ZX}\mathbf{\beta}
$$
Technically, $\mathbf{\beta}$ and $\mathbf{\theta}$ are assumed known because this is a statement about distributions.  In practice, the estimates, $\widehat{\mathbf{\theta}}$ and $\widehat{\beta}$, are plugged in.

A Cholesky decomposition can be written in terms of the lower triangular factor on the left, $\mathbf{L}$, or in terms of $\mathbf{R}$ on the right.  There is a slight technical advantage in evaluating $\mathbf{L}$ in the `MixedModels` package so it is $\mathbf{L}$ that is evaluated and stored.  However, the theory is a bit easier to see in terms of $\mathbf{R}$, which we can obtain as

In [ ]:
UpperTriangular(m1.L')

To evaluate the likelihood,
$$
L(\mathbf{\theta},\mathbf{\beta},\sigma|\mathbf{y}) = \int_\mathbf{u} f_{\mathcal{Y},\mathcal{U}}(\mathbf{y},\mathbf{u})\, d\mathbf{u}
$$
we isolate the part of the joint density that depends on $\mathbf{u}$ and perform a change of variable to
$$
\mathbf{v}=\mathbf{R}_{ZZ}\mathbf{u}+\mathbf{R}_{ZX}\mathbf{\beta}-\mathbf{r}_{Zy} .
$$
From the properties of the multivariate Gaussian distribution
$$
\begin{aligned}
\int_{\mathbf{u}}\frac{1}{(2\pi\sigma^2)^{q/2}}\exp\left(-
\frac{\|\mathbf{R}_{ZZ}\mathbf{u}+\mathbf{R}_{ZX}\mathbf{\beta}-\mathbf{r}_{Zy}\|^2}{2\sigma^2}\right)\,d\mathbf{u}
&=\int_{\mathbf{v}}\frac{1}{(2\pi\sigma^2)^{q/2}}\exp\left(-\frac{\|\mathbf{v}\|^2}{2\sigma^2}\right)|\mathbf{R}_{ZZ}|^{-1}\,d\mathbf{v}\\
&=|\mathbf{R}_{ZZ}|^{-1}
\end{aligned}
$$
from which we obtain the likelihood as
$$
L(\mathbf{\theta},\mathbf{\beta},\sigma)=\frac{|\mathbf{R}_{ZZ}|^{-1}}{(2\pi\sigma^2)^{n/2}}\exp\left(-
\frac{r_{yy}^2 + \|\mathbf{R}_{XX}(\mathbf{\beta}-\widehat{\mathbf{\beta}})\|^2}{2\sigma^2}\right)
$$
If we plug in $\mathbf{\beta}=\widehat{\mathbf{\beta}}$ and take the logarithm we can solve for the estimate of $\sigma^2$, given $\mathbf{\theta}$
$$
\widehat{\sigma^2}=\frac{r_{yy}^2}{n}
$$
which gives the _profiled log-likelihood_, $\ell(\mathbf{\theta}|\mathbf{y})=\log L(\mathbf{\theta},\widehat{\mathbf{\beta}},\widehat{\sigma})$ as
$$
-2\ell(\mathbf{\theta}|\mathbf{y})=2\log(|\mathbf{R}_{ZZ}|) +
    n\left(1+\log\left(\frac{2\pi r_{yy}^2(\mathbf{\theta})}{n}\right)\right)
$$

This may seem complicated but, relative to other formulations of the model, it is remarkably simple.

One of the interesting aspects of this formulation is that it is not necessary to solve for the conditional estimate of $\mathbf{\beta}$ or the conditional modes of the random effects when evaluating the log-likelihood.  The two values needed for the log-likelihood, $2\log(|\mathbf{R}_{ZZ}|)$ and $r_{yy}^2$ are obtained directly from the Cholesky factor.  The logarithm of the determinant,
$$
2\log(|\mathbf{R}_{ZZ}|) = \log(|\mathbf{\Lambda}^\prime\mathbf{Z}^\prime\mathbf{Z}\mathbf{\Lambda}+\mathbf{I}_q|)
$$
is available as

In [ ]:
logdet(m1)

and $r_{yy}^2$ is available as

In [ ]:
pwrss(m1)

which is the square of the element in the lower right corner of either $\mathbf{L}$ or $\mathbf{R}$

In [ ]:
abs2(first(m1.L[Block(3,3)]))

Alternatively, `varest` returns $\widehat{\sigma^2}$

In [ ]:
varest(m1)

This gives the objective function as

In [ ]:
logdet(m1) + dof_residual(m1)*(1 + log(2π * varest(m1)))

One last technical point, the update of the Cholesky factor, $\mathbf{L}$, for a new value of $\mathbf{\theta}$, which generates $\mathbf{\lambda}$ and, hence, $\mathbf{\Lambda}$ can start with the model matrices $\mathbf{Z}$ and $\mathbf{X}$ and the response, $\mathbf{y}$ or it can start with the products, $\mathbf{Z}^\prime\mathbf{Z}$, etc.  The package uses the second approach which is more efficient when the number of observations is large relative to the number of random effects.  The non-redundant products are stored in the `A` field.

In [ ]:
Symmetric(m1.A, :L)

Because the experiment is _balanced_, in the sense that each subject's reaction time is measured the same number of times and after the same number of days of sleep deprivation, the diagonal blocks in $\mathbf{Z}^\prime\mathbf{Z}$ are repetitions of one another.  The number in the lower right-hand corner of `A` is $\mathbf{y}^\prime\mathbf{y}$ or

In [ ]:
sum(abs2, sleepstudy.Reaction)

## Mixed-models and shrinkage of estimates

[John Tukey](https://en.wikipedia.org/wiki/John_Tukey) characterized the _regularization_ or _shrinkage_ aspects of mixed-effects models as _borrowing strength_ from the estimates for other subjects in the experiment.  The penalty term in the penalized least squares calculation has the effect of shrinking an individual's coefficients in the predictor back toward the global estimates.

In [ ]:
R"""
df <- coef(lmList(Reaction ~ Days | Subject, sleepstudy))
fm2 <- lmer(Reaction ~ Days + (Days|Subject), sleepstudy)
fclow <- subset(df, `(Intercept)` < 251)
fchigh <- subset(df, `(Intercept)` > 251)
cc1 <- as.data.frame(coef(fm2)$Subject)
names(cc1) <- c("A", "B")
df <- cbind(df, cc1)
ff <- fixef(fm2)
with(df,
     print(xyplot(`(Intercept)` ~ Days, aspect = 1,
                  x1 = B, y1 = A,
                  panel = function(x, y, x1, y1, subscripts, ...) {
                      panel.grid(h = -1, v = -1)
                      x1 <- x1[subscripts]
                      y1 <- y1[subscripts]
                      larrows(x, y, x1, y1, type = "closed", length = 0.1,
                              angle = 15, ...)
                      lpoints(x, y,
                              pch = trellis.par.get("superpose.symbol")$pch[2],
                              col = trellis.par.get("superpose.symbol")$col[2])
                      lpoints(x1, y1,
                              pch = trellis.par.get("superpose.symbol")$pch[1],
                              col = trellis.par.get("superpose.symbol")$col[1])
                      lpoints(ff[2], ff[1],
                              pch = trellis.par.get("superpose.symbol")$pch[3],
                              col = trellis.par.get("superpose.symbol")$col[3])
                      ltext(fclow[,2], fclow[,1], row.names(fclow),
                            adj = c(0.5, 1.7))
                      ltext(fchigh[,2], fchigh[,1], row.names(fchigh),
                            adj = c(0.5, -0.6))
                  },
                  key = list(space = "top", columns = 3,
                  text = list(c("Mixed model", "Within-group", "Population")),
                  points = list(col = trellis.par.get("superpose.symbol")$col[1:3],
                  pch = trellis.par.get("superpose.symbol")$pch[1:3]))
                  )))
""";

Comparing this plot to the original data plot with the lines from the various fits superimposed

In [ ]:
R"""
print(xyplot(Reaction ~ Days | Subject, sleepstudy, aspect = "xy",
             layout = c(9,2), type = c("g", "p", "r"),
             coef.list = df[,3:4],
             panel = function(..., coef.list) {
                 panel.xyplot(...)
                 panel.abline(as.numeric(coef.list[packet.number(),]),
                              col.line = trellis.par.get("superpose.line")$col[2],
                              lty = trellis.par.get("superpose.line")$lty[2]
                              )
                 panel.abline(fixef(fm2),
                              col.line = trellis.par.get("superpose.line")$col[4],
                              lty = trellis.par.get("superpose.line")$lty[4]
                              )
             },
             index.cond = function(x,y) coef(lm(y ~ x))[1],
             xlab = "Days of sleep deprivation",
             ylab = "Average reaction time (ms)",
             key = list(space = "top", columns = 3,
             text = list(c("Within-subject", "Mixed model", "Population")),
             lines = list(col = trellis.par.get("superpose.line")$col[c(2:1,4)],
             lty = trellis.par.get("superpose.line")$lty[c(2:1,4)]))))
""";

shows that the fits for those subjects whose data shows a strong linear trend (e.g. 308, 309, 310, 337) are not changed that much.  But those whose data does not define a line well (e.g. 330, 331) are shrunk toward the global fit.

In [ ]:
using MixedModelsTutorials
MixedModelsTutorials.tutorial_footer(WEAVE_ARGS[:folder],WEAVE_ARGS[:file])